# Evaluate Berkeley Predictions using Deep CBN

    ## ER Alpha

In [39]:
import pandas as pd
# evaluate what Tox21 and Toxcast data we have for Berkeley data
tox21 = pd.read_csv('../Data/tox21.csv')
toxcast = pd.read_csv('../Data/toxcast.csv')
# Display the columns in tox21 and toxcast  
print("Tox21 columns:", tox21.columns.tolist())
print("Toxcast columns:", toxcast.columns.tolist())

# search for PPAR contained in Tox21 and Toxcast columns
PPARA_tox21 = [col for col in tox21.columns if 'PPAR' in col]
PPARA_toxcast = [col for col in toxcast.columns if 'PPAR' in col]

PPARA_tox21
PPARA_toxcast


Tox21 columns: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'mol_id', 'smiles']
Toxcast columns: ['smiles', 'ACEA_T47D_80hr_Negative', 'ACEA_T47D_80hr_Positive', 'APR_HepG2_CellCycleArrest_24h_dn', 'APR_HepG2_CellCycleArrest_24h_up', 'APR_HepG2_CellCycleArrest_72h_dn', 'APR_HepG2_CellLoss_24h_dn', 'APR_HepG2_CellLoss_72h_dn', 'APR_HepG2_MicrotubuleCSK_24h_dn', 'APR_HepG2_MicrotubuleCSK_24h_up', 'APR_HepG2_MicrotubuleCSK_72h_dn', 'APR_HepG2_MicrotubuleCSK_72h_up', 'APR_HepG2_MitoMass_24h_dn', 'APR_HepG2_MitoMass_24h_up', 'APR_HepG2_MitoMass_72h_dn', 'APR_HepG2_MitoMass_72h_up', 'APR_HepG2_MitoMembPot_1h_dn', 'APR_HepG2_MitoMembPot_24h_dn', 'APR_HepG2_MitoMembPot_72h_dn', 'APR_HepG2_MitoticArrest_24h_up', 'APR_HepG2_MitoticArrest_72h_up', 'APR_HepG2_NuclearSize_24h_dn', 'APR_HepG2_NuclearSize_72h_dn', 'APR_HepG2_NuclearSize_72h_up', 'APR_HepG2_OxidativeStress_24h_up', 'APR_HepG2_OxidativeStress

['ATG_PPARa_TRANS_dn',
 'ATG_PPARa_TRANS_up',
 'ATG_PPARd_TRANS_up',
 'ATG_PPARg_TRANS_up',
 'NVS_NR_hPPARa',
 'NVS_NR_hPPARg',
 'TOX21_PPARd_BLA_Agonist_viability',
 'TOX21_PPARd_BLA_Antagonist_ch1',
 'TOX21_PPARd_BLA_agonist_ch1',
 'TOX21_PPARd_BLA_agonist_ch2',
 'TOX21_PPARd_BLA_agonist_ratio',
 'TOX21_PPARd_BLA_antagonist_ratio',
 'TOX21_PPARd_BLA_antagonist_viability',
 'TOX21_PPARg_BLA_Agonist_ch1',
 'TOX21_PPARg_BLA_Agonist_ch2',
 'TOX21_PPARg_BLA_Agonist_ratio',
 'TOX21_PPARg_BLA_Antagonist_ch1',
 'TOX21_PPARg_BLA_antagonist_ratio',
 'TOX21_PPARg_BLA_antagonist_viability']

In [40]:
# This script merges the Berkeley data with chemical IDs and filters for reliable activity data.

### Import Berkeley data
# ERa, PPARg, AR
ER = pd.read_excel('../Data/berkeley/ERA.xlsx')
PPARg = pd.read_excel('../Data/berkeley/PPARg.xlsx')
AR = pd.read_excel('../Data/berkeley/AR.xlsx')

import pyreadr
chemical_IDs_result = pyreadr.read_r('../Data/berkeley/chemical_IDs.rds')
# Convert the chemical_IDs OrderedDict to a DataFrame
chemical_IDs = list(chemical_IDs_result.values())[0]  # Extract the actual DataFrame

# Merge on 'DTXSID'
merged_ER = pd.merge(ER[['DTXSID', 'Actvity_AGO', 'AD_AGO']], chemical_IDs[['DTXSID', 'SMILES']], on='DTXSID', how='left')
merged_PPARg = pd.merge(PPARg[['DTXSID', 'Actvity_AGO', 'AD_AGO']], chemical_IDs[['DTXSID', 'SMILES']], on='DTXSID', how='left')
merged_AR = pd.merge(AR[['DTXSID', 'Actvity_AGO', 'AD_AGO']], chemical_IDs[['DTXSID', 'SMILES']], on='DTXSID', how='left')

# filter for AD_AGO == "reliable"
merged_ER = merged_ER[merged_ER['AD_AGO'] == 'reliable']
merged_PPARg = merged_PPARg[merged_PPARg['AD_AGO'] == 'reliable']
merged_AR = merged_AR[merged_AR['AD_AGO'] == 'reliable']

#recode Actvity_AGO in merged_df datafram
def recode_activity(activity):
    if activity == 'act.':
        return 1
    elif activity == 'inact.':
        return 0
    
# apply recode function to 'Actvity_AGO' column
merged_ER['NR-ER'] = merged_ER['Actvity_AGO'].apply(recode_activity)
merged_PPARg['NR-PPAR-gamma'] = merged_PPARg['Actvity_AGO'].apply(recode_activity)
merged_AR['NR-AR'] = merged_AR['Actvity_AGO'].apply(recode_activity)

# merge all three DataFrames on 'DTXSID'
merged_df = pd.merge(merged_ER, merged_PPARg, on='DTXSID', how='left')
merged_df = pd.merge(merged_df, merged_AR, on='DTXSID', how='left')

# Rename columns for clarity    
merged_df = merged_df.rename(columns={'SMILES_x': 'SMILES'})

# Remove ducplicate SMILES columns
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Select relevant columns and rename them
merged_df = merged_df[['DTXSID', 'SMILES', 'NR-ER', 'NR-PPAR-gamma', 'NR-AR']]

# Drop rows with NaN values in the SMILES columns
merged_df = merged_df.dropna(subset=['SMILES'])

# Display or check result
print(merged_df.head())

          DTXSID                                             SMILES  NR-ER  \
0  DTXSID4020614  CC(=O)OC1CCC2C3CCC4(C(C3CCC2=C1)CCC4(C#C)OC(=O...    1.0   
1  DTXSID7037717  C1=CC(=CC=C1C(C2=CC=C(C=C2)O)(C(F)(F)F)C(F)(F)F)O    1.0   
2  DTXSID0038887  C1=CC=C2C(=C1)C(=O)OC23C4=C(C=C(C=C4)O)OC5=C3C...    1.0   
3  DTXSID8022408  C1=CC=C2C(=C1)C(OS2(=O)=O)(C3=CC=C(C=C3)O)C4=C...    1.0   
4  DTXSID0022436         CC(CCC(=O)O)(C1=CC=C(C=C1)O)C2=CC=C(C=C2)O    1.0   

   NR-PPAR-gamma  NR-AR  
0            NaN    NaN  
1            NaN    NaN  
2            NaN    NaN  
3            NaN    NaN  
4            NaN    NaN  


In [ ]:
## Import Functions
from train_deep_cbn_fnx import train_deep_cbn, calculate_roc_auc, plot_confusion_matrix, process_multiple_targets,  predict_with_models, label_smiles

#### Train models for multiple targets ###
# Target columns to process from dataset
#target_cols = ['NR-PPAR-gamma', 'NR-AhR', 'SR-p53']
target_cols = ['NR-PPAR-gamma', 'NR-AR', 'NR-ER']
dataset_path = '../Data/tox21.csv' #target dataset
smiles_col = 'smiles' #annotate SMILES column
n_epochs = 100 # keep this at 100 except for beta-testing

# Call the function
results_df, models_dict = process_multiple_targets(dataset_path, target_cols, smiles_col, n_epochs)

# Display the results dataframe
print(results_df)


Processing target column: NR-PPAR-gamma


c:\Users\Scott.Coffin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'bi_level_routing_attention_16', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
c:\Users\Scott.Coffin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'bi_level_routing_attention_17', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
c:\Users\Scott.Coffin\AppData\Local\Prog

Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 363ms/step - accuracy: 0.8008 - auc: 0.8262 - f1_score: 0.4658 - loss: 0.6849 - precision: 0.8008 - recall: 0.8008
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 354ms/step - accuracy: 0.6137 - auc: 0.6276 - f1_score: 0.4189 - loss: 0.6836 - precision: 0.6137 - recall: 0.6137
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 346ms/step - accuracy: 0.7038 - auc: 0.7592 - f1_score: 0.4604 - loss: 0.6571 - precision: 0.7038 - recall: 0.7038
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 393ms/step - accuracy: 0.5807 - auc: 0.5777 - f1_score: 0.4099 - loss: 0.6754 - precision: 0.5807 - recall: 0.5807
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 358ms/step - accuracy: 0.7195 - auc: 0.7817 - f1_score: 0.4766 - loss: 0.7084 - precision: 0.7195 - recall: 0.7195
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 368ms/step - accuracy: 0.7787 - auc: 0.8455 - f1_score: 0.5035 - loss: 0.6508 - precision: 0.7787 - recall: 0.7787
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 359ms/step - ac

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

# Load dataset for predictions
new_data = merged_df
smiles_list = new_data['SMILES']

    # Dictionary for converting SMILES characters to numbers
smiles_dict = {
    "#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2,
    "1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6,
    "9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43,
    "D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13,
    "O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51,
    "V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\\": 20, "a": 55, "c": 56,
    "b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60,
    "l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64,
    " ": 65, ":": 66, ",": 67, "p": 68, "j": 69, "*": 70
    }


def label_smiles(line, MAX_SMI_LEN, smi_ch_ind):
    X = np.zeros(MAX_SMI_LEN, dtype=int)
    for i, ch in enumerate(line[:MAX_SMI_LEN]):
        if ch in smi_ch_ind:
            X[i] = smi_ch_ind[ch]
    return X

X_new = np.array([label_smiles(str(s), 100, smiles_dict) for s in smiles_list])
X_new = to_categorical(X_new, num_classes=71)

target_cols = ['NR-PPAR-gamma', 'NR-AR', 'NR-ER']
X_test_dict = {}
y_test_cat_dict = {}
valid_indices_dict = {}

# only drop NAs for target column being predicted upon
for target in target_cols:
    # Keep only non-NA rows for this target
    valid_idx = new_data[target].notna()
    
    # Save indices (optional: useful for re-aligning predictions later)
    valid_indices_dict[target] = valid_idx

    # Prepare X and y for this target
    smiles_subset = smiles_list[valid_idx]
    X_target = np.array([label_smiles(str(s), 100, smiles_dict) for s in smiles_subset])
    X_target = to_categorical(X_target, num_classes=71)

    # Store test features
    X_test_dict[target] = X_target

    # Convert valid y values to categorical
    y_valid = new_data.loc[valid_idx, target].astype(int)
    y_test_cat_dict[target] = to_categorical(y_valid, num_classes=2)

predictions_df = predict_with_models(models_dict, X_test_dict, y_test_cat_dict)
print(predictions_df)

Predicting for target column: NR-PPAR-gamma
113/113 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
Accuracy for NR-PPAR-gamma: 0.9875
AUC for NR-PPAR-gamma: 0.5613
Confusion Matrix for NR-PPAR-gamma:
[[3541    0]
 [  45    0]]

Predicting for target column: NR-AR
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step
Accuracy for NR-AR: 0.0233
AUC for NR-AR: 0.6319
Confusion Matrix for NR-AR:
[[   0 3482]
 [   0   83]]

Predicting for target column: NR-ER
1509/1509 ━━━━━━━━━━━━━━━━━━━━ 47s 31ms/step
Accuracy for NR-ER: 0.8856
AUC for NR-ER: 0.2750
Confusion Matrix for NR-ER:
[[42741     0]
 [ 5521     0]]

      target_col  accuracy       auc         confusion_matrix
0  NR-PPAR-gamma  0.987451  0.561307     [[3541, 0], [45, 0]]
1          NR-AR  0.023282  0.631861     [[0, 3482], [0, 83]]
2          NR-ER  0.885604  0.274983  [[42741, 0], [5521, 0]]
